In [ ]:
# Imports

import numpy as np
from tensorflow.keras.datasets import fashion_mnist, mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import seaborn as sn
import wandb

In [ ]:
# Creating train and test data

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

X_train_new = X_train.reshape(
    (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test_new = X_test.reshape(
    (X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

In [ ]:
# The RBN class

def sig(z):
    return np.where(z > 0, 1 / (1 + np.exp(-z)), np.exp(z) / (1 + np.exp(z)))

class RBNClassifier:
    
    def __init__(self, hidden_layer_dim = 64, n_classes = 10, k = 200, r = 10, learning_rate = 0.01, epochs = 1, sampling = 'Gibbs'):
        self.input_dim = None
        self.hidden_layer_dim = hidden_layer_dim
        self.n_classes = n_classes
        self.k = k
        self.r = 10
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.sampling = sampling
        self.clf = RandomForestClassifier()
        
        if(self.sampling != 'Gibbs'): # Contrastive divergence takes a point estimate
            self.r = 1
    
    def prob_v(self, h):
        return sig(np.matmul(h, self.w.T) + self.b)
    
    def prob_h(self, x):
        return sig(np.matmul(x, self.w) + self.c)
    
    def get_hidden(self, X):
        probs = sig(np.sign(np.matmul(X, self.w) + self.c))
        return np.asnumpy(np.random.binomial(1, probs, probs.shape))
    
    def simulate_markov(self, X):
        v = X
        hid = np.zeros((X.shape[0], self.hidden_layer_dim))
        
        for t in range(0, self.k):
            hid = np.random.binomial(1, self.prob_h(v), hid.shape)
            v = np.random.binomial(1, self.prob_v(hid), v.shape)
        
        v_list = []
        
        for t in range(0, self.r):
            hid = np.random.binomial(1, self.prob_h(v), hid.shape)
            v = np.random.binomial(1, self.prob_v(hid), v.shape)
            v_list.append(v)
            
        return np.array(v_list)
    
    def single_update(self, x, all_v):
    
        transf_all_v = sig(np.matmul(all_v, self.w) + self.c)
        mat_all_v = np.matmul(all_v.reshape((self.r, self.input_dim, 1)), transf_all_v.reshape((self.r, 1, self.hidden_layer_dim)))
        
        transf_x = sig(np.matmul(x, self.w) + self.c)
        mat_x = np.matmul(x.reshape((self.input_dim, 1)), transf_x.reshape((1, self.hidden_layer_dim)))
        
        self.w += self.learning_rate * (mat_x - mat_all_v.mean(axis=0))
        self.b += self.learning_rate * (x - all_v.mean(axis=0))
        self.c += self.learning_rate * (transf_x - transf_all_v.mean(axis=0))
        
    def weights_init(self):
        self.b = np.zeros((self.input_dim,))
        self.c = np.zeros((self.hidden_layer_dim,))
        self.w = np.zeros((self.input_dim, self.hidden_layer_dim))
                    
    def fit(self, X, y, X_t, y_t):
        self.input_dim = X.shape[1]
        self.weights_init()
        
        num_bars = 30
        
        for ep in range(self.epochs):
            print('Epoch ', ep + 1, '/', self.epochs)
            
            batch_size = 64
            n_inputs = X.shape[0]
            n_bars = 30
            
            for ii in range(0, n_inputs, batch_size):
                X_curr = X[ii:ii + batch_size]
            
                all_v = self.simulate_markov(X_curr).swapaxes(0, 1)
                
                for jj, x in enumerate(X_curr):
                    self.single_update(x, all_v[jj])
                
                curr = (num_bars * (ii + 1)) // n_inputs
                print('\r[' + '=' * curr + '.' * (num_bars - curr) + ']', end = '')
            print()
            
            hid = self.get_hidden(X)
            self.clf.fit(hid, np.asnumpy(y))
            
            hid_t = self.get_hidden(X_t)
            
            test_acc = self.clf.score(hid_t, y_t)
            test_loss = -self.clf.predict_proba(hid_t)[range(X_t.shape[0]), y_t].sum()
        
            print('Test Accuracy : ', test_acc, ', Test Loss : ', test_loss)
    
    def predict(self, X):
        hid = self.get_hidden(X)
        return self.predict(hid)

In [ ]:
clf = RBNClassifier()
clf.fit(X_train_new, Y_train, X_test_new, np.asnumpy(Y_test))